# Setting some Thresholds for Rainfall

This notebook looks at initial thoughts around setting a rainfall trigger using the landfall date or the date after landfall.

In [1]:
%load_ext jupyter_black
import os
import pandas as pd
import numpy as np
from pathlib import Path
import geopandas as gpd
import warnings
import plotly.express as px
import plotly.graph_objects as go
from datetime import timedelta

In [2]:
warnings.filterwarnings("ignore")
AA_DATA_DIR = Path(os.getenv("AA_DATA_DIR"))
AA_DATA_DIR_NEW = Path(os.getenv("AA_DATA_DIR_NEW"))

ADMS = ["Sofala", "Inhambane", "Nampula", "Zambezia"]

In [3]:
emdat_path = (
    AA_DATA_DIR_NEW
    / "private"
    / "processed"
    / "glb"
    / "emdat"
    / "emdat-tropicalcyclone-2000-2022-processed-sids.csv"
)

ibtracs_path = (
    Path(AA_DATA_DIR)
    / "public"
    / "raw"
    / "glb"
    / "ibtracs"
    / "IBTrACS.SI.list.v04r01.points/IBTrACS.SI.list.v04r01.points.shp"
)

adm1_path = (
    AA_DATA_DIR
    / "public"
    / "raw"
    / "moz"
    / "cod_ab"
    / "moz_admbnda_adm1_ine_20190607.shp"
)

inam_path = (
    AA_DATA_DIR
    / "private"
    / "raw"
    / "moz"
    / "inam"
    / "OCHA_blended_moz"
    / "OCHA_blended_moz"
)
df_emdat = pd.read_csv(emdat_path)
gdf_adm1 = gpd.read_file(adm1_path)

gdf_adm1_sel = gdf_adm1[gdf_adm1.ADM1_PT.isin(ADMS)]
df_emdat = df_emdat[
    df_emdat.iso3 == "MOZ"
]  # Just want the impacts to Mozambique

In [4]:
imerg_path = (
    Path(AA_DATA_DIR)
    / "public"
    / "processed"
    / "moz"
    / "daily_imerg_cyclone_landfall.csv"
)

df_imerg = pd.read_csv(imerg_path)
df_imerg["date"] = pd.to_datetime(df_imerg["date"])

In [5]:
df_imerg

,Unnamed: 0,storm,date,time_step,radius,median_precip
0,0,FAVIO,2007-02-20 10:00:00,-2,50,0.602754
1,1,FAVIO,2007-02-20 10:00:00,-2,100,0.307900
2,2,FAVIO,2007-02-20 10:00:00,-2,150,0.072822
3,3,FAVIO,2007-02-20 10:00:00,-2,200,0.004476
4,4,FAVIO,2007-02-20 10:00:00,-2,250,0.000000
...,...,...,...,...,...,...
1675,1755,FILIPO,2024-03-17 05:00:00,5,300,0.589248
1676,1756,FILIPO,2024-03-17 05:00:00,5,350,0.306609
1677,1757,FILIPO,2024-03-17 05:00:00,5,400,0.260966
1678,1758,FILIPO,2024-03-17 05:00:00,5,450,0.260798


## Get the rainfall per day

In [8]:
# Calculate the mean of median_precip_250km for each time_step
mean_precip = (
    df_imerg[df_imerg["radius"] == 250]
    .groupby("time_step")["median_precip"]
    .mean()
    .reset_index()
)

In [11]:
# Plot with Plotly
fig = px.line(
    mean_precip,
    x="time_step",
    y="median_precip",
    title="Mean Median Precipitation (250km) by Time Step",
    labels={
        "time_step": "Days to Cyclone Landfall",
        "median_precip": "Median Precipitation in 250km radius",
    },
)
fig.update_traces(mode="markers+lines")
fig.show()

## Rainfall by Storm

In [15]:
# Filter data for time_step = 0
df_time_step_0 = df_imerg[
    (df_imerg["time_step"] == 0) & (df_imerg["radius"] == 250)
]

# Sort the values by median_precip_250km in descending order
df_time_step_0_sorted = df_time_step_0.sort_values(
    by="median_precip", ascending=False
)

# Plot with Plotly
fig = px.bar(
    df_time_step_0_sorted,
    x="median_precip",
    y="storm",
    title="Rainfall on Landfall Date for Each Storm",
    labels={
        "storm": "Storm",
        "median_precip": "Median Precipitation (in 250km radius)",
    },
    orientation="h",
)
fig.update_layout(yaxis={"categoryorder": "total ascending"}, height=600)
fig.show()

In [17]:
# Plot with Plotly
fig = px.bar(
    df_time_step_0_sorted,
    x="median_precip",
    y="storm",
    title="Rainfall on the day after Landfall for Each Storm",
    labels={
        "storm": "Storm",
        "median_precip": "Median Precipitation (in 250km radius)",
    },
    orientation="h",
)
fig.update_layout(yaxis={"categoryorder": "total ascending"}, height=600)
fig.show()

## Setting threshold to 40mm on landfall date

In [18]:
df_time_step_0[df_time_step_0["median_precip"] >= 40]

,Unnamed: 0,storm,date,time_step,radius,median_precip
1304,1304,ANA,2022-01-24 08:00:00,0,250,60.963920
1384,1464,GOMBE,2022-03-11 00:30:00,0,250,79.903120
1544,1624,FREDDY,2023-03-11 17:30:00,0,250,46.478664


In [20]:
## INAM Data
## Data shared by Gabriela
## reading in the file and merging with df_imerg to assign storms
# removing those before 2007 and after 2021
df_restricted = df_imerg[
    (df_imerg["date"] >= pd.to_datetime("2007-01-01"))
    & (df_imerg["date"] <= pd.to_datetime("2021-12-31"))
]
df_rest_time_1 = df_restricted[df_restricted["time_step"] == 1]

# Sort the values by median_precip_250km in descending order
df_rest_time_1_sorted = df_rest_time_1.sort_values(
    by="median_precip", ascending=False
)

# Plot with Plotly
fig = px.bar(
    df_rest_time_1_sorted,
    x="median_precip",
    y="storm",
    title="Rainfall on the day after Landfall for Each Storm",
    labels={
        "storm": "Storm",
        "median_precip": "Median Precipitation (in 250km radius)",
    },
    orientation="h",
)
fig.update_layout(yaxis={"categoryorder": "total ascending"}, height=600)
fig.show()